# Basic protein modeling in Rosetta

Rosetta is a software suite for working with models of proteins (or other molecules, like small molecule ligands and RNA).

We can access all of Rosetta's functions using the `pyrosetta` python package.

In [99]:
from pyrosetta import *
pyrosetta.init()

PyRosetta-4 2020 [Rosetta PyRosetta4.Release.python38.mac 2020.28+release.8ecab77aa50ac1301efe53641e07e09ac91fee3b 2020-07-07T16:41:06] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python38.mac r260 2020.28+release.8ecab77aa50 8ecab77aa50ac1301efe53641e07e09ac91fee3b http://www.pyrosetta.org 2020-07-07T16:41:06
core.init: command: PyRosetta -ex1 -ex2aro -database /Users/cjmathy/.pyenv/versions/3.8.4/lib/python3.8/site-packages/pyrosetta-2020.28+release.8ecab77aa50-py3.8-macosx-10.15-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-334859078 seed_offset=0 real_seed=-334859078
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-334859078 RG_type=mt19937


## Storing structure information in Rosetta with the Pose class

Protein structures are stored in text files that contain the sequence information and the positions of each atom in cartesian space. We'll work with the `PDB` format , but there are other useful ones that Rosetta can use, such as the newer `mmCIF` format which accommates much larger models.

After reading in a `PDB` file, Rosetta stores the structure as an object of the `Pose` class. It contains the atoms in the structure and the chemical connectivity. Additionally, it can store energies for each residue, any constraints we want to apply to the structure, and annotations from the original PDB. To manipulate a structure with Rosetta, we make operations on the `Pose` object.

We read in a `PDB` file with the function `pose_from_pdb` in the `pyrosetta` package. Rosetta will print out lots of information about what it is parsing from the `PDB` file, choices it makes to pick a final representation, and compute the score (in Rosetta Energy Units or REU, something like a ΔG) of the structure using a default score function.

We will work with the PDB 1QYS, which is Top7, a novel fold of protein designed using Rosetta!

In [214]:
pose = pyrosetta.pose_from_pdb('1qys.pdb')

core.import_pose.import_pose: File '1qys.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading Selenium SE from MSE as SD from MET
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue LYS 13
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue LYS 13
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CE  on residue LYS 13
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NZ  on 

We can get lots of useful information from this `Pose` object, including:

- The structure's sequence
- The number of chains
- The residue numbering (when residues are missing from a structure, the `Pose` numbering will not match the `PDB` numbering, as the `Pose` numbering always starts at 1.
- The atomic position (X,Y,Z) coordinates of any atom, such as the alpha-carbons of two residues (and therefore the distance between them). The units are angstroms.
- The energy of individual residues
- The energy of the structure

In [101]:
# sequence
pose.sequence()

'DIQVQVNIDDNGKNFDYTYTVTTESELQKVLNELMDYIKKQGAKRVRISITARTKKEAEKFAAILIKVFAELGYNDINVTFDGDTVTVEGQL'

In [102]:
# chain and numbering info
print(pose.pdb_info())

PDB file name: 1qys.pdb
 Pose Range  Chain    PDB Range  |   #Residues         #Atoms

0001 -- 0092    A 0003  -- 0094  |   0092 residues;    01477 atoms
                           TOTAL |   0092 residues;    01477 atoms



In [103]:
# CA positions and distance
CA_1 = pose.residue(1).xyz("CA")
CA_2 = pose.residue(2).xyz("CA")

print(CA_1)
print(CA_2)

     -3.061000000000000       18.22800000000000       17.12200000000000
    -0.8230000000000000       15.56200000000000       15.56800000000000


In [104]:
# Distance between the two residues, using the function norm()
print((atom1 - atom2).norm())

16.55723980619958


In [105]:
# the energy terms for the 5th residue
pose.energies().show(5)

core.scoring.Energies: E               fa_atr        fa_rep        fa_sol  fa_intra_repfa_intra_sol_x   lk_ball_wtd       fa_elec     pro_close   hbond_sr_bb   hbond_lr_bb   hbond_bb_sc      hbond_sc     dslf_fa13         omega        fa_dun       p_aa_pp yhh_planarity           ref   rama_prepro
core.scoring.Energies: E(i)   5         -5.98          0.87          6.24          3.71          0.49          0.58         -2.63          0.00          0.00          0.00          0.00         -0.60          0.00          0.09         21.42          0.46          0.00         -1.45          0.09


In [106]:
# Structure energy
scorefxn = get_fa_scorefxn()  # a score function is just the equation that relates
scorefxn(pose)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015


206.2147741464653

A good resource for seeing everything the `Pose` object has to offer is Figure 19.2 in the following paper describing Rosetta: 
    
- Leaver-Fay, A., Tyka, M., Lewis, S. M., Lange, O. F., Thompson, J., Jacak, R., ... & Bradley, P. (2011). ROSETTA3: an object-oriented software suite for the simulation and design of macromolecules. Methods in enzymology, 487, 545-574.
    https://www.sciencedirect.com/science/article/pii/B9780123812704000196

## Visualizing your pose with PyMOL

While Rosetta doesn't have its own GUI for viewing protein models, it is set up for easy linking to PyMOL. You should have received the script `PyMOL-RosettaServer.python3.py`. To start a connection between PyRosetta and PyMOL, start PyMOL and run `run ~/PyRosetta/PyMOL-RosettaServer.python3.py`. If the link is active, you should see "PyMOL <---> PyRosetta link started!" in your PyMOL console.
    
NOTE: You can also make a file `.pymolrc` in your home directory and put that line of code there for it to run automatically when you initialize PyMOL. 
    
Once the link is set up, we can use a new class, the `PyMOLMover` to send the pose to PyMOL. A `Mover` is a class of Rosetta objects that operate on / alter `Pose` objects.

In [144]:
pmm = PyMOLMover()
pmm.apply(pose)

You should now see Top7 in your PyMOL window.

Let's color each residue according to an energy term, `fa_sol`. This term accounts for the energetic penalty or favorability of a given side chain's solvent accessibility in the structure.

In [145]:
pmm.label_energy(pose, 'fa_sol')

You should see a striped pattern along the beta sheet. This is expected, as residues whose side chains point inwards towards the core are more blue (more negative/energetically favorable), while residues with solvent-exposed side chains are more red, as they pay a solvation penalty.

This favorable tertiary structure pattern of residues - a hydrophobic core - was incorporated intentionally to enable folding into a stable protein.

If you'd like to examine the way the side chains fit together within the core, you can see them by typing the following command into the PyMOL console: `show sticks, not (name c+n+o) and not hydro`

## Relaxing a structure

You may have noticed that when we scored the pose above, the energy was positive (~200 REU), which means Rosetta thinks our pose is pretty unfavorable. Often, the bond lengths and angles of structures from the PDB are not ideal according to Rosetta. This is for a variety of reasons, both physical (crystal contacts introduce a bias, for example) and algorithmic (the refinement forcefields used to solve crystal structures and the score function in Rosetta each have their distinct biases). Typically when manipulating or analyzing a new PDB, we first optimize the structure for the Rosetta score function using a protocol known as `Relax`.

Relaxing a structure consists of trying many slight perturbations to the backbone bonds ("minimization") as well as trying other sidechain rotamers ("repacking"). We will learn more about these protocols later, but for now let's just run the `Relax` code and see how it changes our structure.

This run should take about 1 minute, and will spit out a lot of log information, as it iteratively. perturbs and re-scores the structure.

In [142]:
# make a copy of the pose
pose_relax = pose.clone()
pose_relax.pdb_info().name('1qys_relaxed')

# create a Mover that can run Relax on our pose
relax_mover = pyrosetta.rosetta.protocols.relax.FastRelax()
relax_mover.set_scorefxn(scorefxn)                   # we use the default score function, which we initialized earlier
relax_mover.constrain_relax_to_start_coords(True)    # we want to stay close to the starting coordinates, just refining a bit

# run relax
relax_mover.apply(pose_relax)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
protocols.relax.FastRelax: CMD: repeat  206.215  0  0  0.55
protocols.relax.FastRelax: CMD: coord_cst_weight  206.215  0  0  0.55
protocols.relax.FastRelax: CMD: scale:fa_rep  7.09245  0  0  0.022
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 2042 rotamers at 92 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  -225.455  0  0  0.022
protocols.relax.FastRelax: CMD: scale:fa_rep  -218.369  0  0  0.02805
protocols.relax.FastRelax: CMD: min  -410.541  0.971168  0.971168  0.02805
protocols.relax.FastRelax: CMD: coord_cst_weight  -410.541  0.971168  0.971168  0.02805
protocols.relax.FastRelax: CMD: scale:fa_rep  -218.021  0.971168  0.971168  0.14575
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 2413 rotamers at 92 positions.
core.pack.interaction_graph.i

In [146]:
pmm.apply(pose_relax)

These two poses likely look very different, but that is in fact because the relaxed pose is translated and rotated in the PyMOL coordinate system. To really compare the two structures, run the following command in the PyMOL console `align 1qys.pdb, 1qys_relaxed`. This will align the original structure to the relaxed structure. The two structures should seem much more similar. PyMOL calculates the degree of difference between the two models after alignment, reported as a Root Mean Squared Deviation (RMSD). It should be about 0.7 or 0.8.

We can also compute an RMSD metric in Rosetta - the function we will use only computes distances between matched alpha-carbons between the two structures:

In [148]:
pyrosetta.rosetta.core.scoring.CA_rmsd(pose, pose_relax) 

0.78050297498703

We can also see if the side chains have moved around. Type the following command into your PyMOL console: `show sticks, not (name c+n+o) and not hydro`. You should see that some of the residues have different rotamers for their side chains, such a Lys31 or Asp38.

Finally, let's see if the `Relax` protocol did improve the score (energy) of the structure, as intended:

In [149]:
initial_score = scorefxn(pose)
relaxed_score = scorefxn(pose_relax)

print(initial_score)
print(relaxed_score)
print(relaxed_score - initial_score)

206.2147741464653
-304.9643401505464
-511.17911429701167


Are you surprised by the large difference in score? Despite the overall change in structure being less than an Angstrom (using the RMSD metrics), there was a change of about ~500 REU! This was all from slight optimizations in protein backbone angles and side chain rotamer interactions. The score function is parameterized for 1 REU to be on the order of 0.1-1 kcal/mol, so this is a sizeable difference.

## Mutating a pose

Protein design involves iteratively changing a protein sequence with new mutations and seeing if the resulting structure scores better in a desired metric (usually energy). Let's try this out by mutating a residue in the relaxed Top7 pose and seeing how this changes the overall energy. We will use a convenient function, `mutate_residue` to mutate a hydrophobic residue (Val 8) in the core of protein to a less favorable polar residue (Thr). We will need to get the pose number for residue 8 first.

In [213]:
pose_V8T = pose_relax.clone()
pose_V8T.pdb_info().name('1qys_V8T')

res8 = pose_V8T.pdb_info().pdb2pose(chain='A',res=8)
pyrosetta.toolbox.mutate_residue(pack_or_pose=pose_V8T,
                                 mutant_position=res8,
                                 mutant_aa='T',
                                 pack_radius=0)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 108 rotamers at 1 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph


We can confirm that we mutated the residue correctly by querying the name of the residue at that position in both our original and mutated pose

In [206]:
# confirm we mutated correctly
print(pose_relax.residue(res8).name())
print(pose_V8T.residue(res8).name())

VAL
THR


Now let's score the two poses

In [207]:
mutated_score = scorefxn(pose_V8T)

print(mutated_score - relaxed_score)

4.795649293731344


We increased the score by ~5 REU, so Rosetta agrees with our intuition that the hydrophobic sidechain Val was preferred over Thr at that core position.

We can look at the individual score terms to see which physical force accounts for this change in score

In [211]:
print('Val8 terms, relaxed structure:\n------------')
pose_relax.energies().show(res8)
print('\nThr8 terms, mutated structure:\n------------')
pose_V8T.energies().show(res8)


Val8 terms, relaxed structure:
------------
core.scoring.Energies: E               fa_atr        fa_rep        fa_sol  fa_intra_repfa_intra_sol_x   lk_ball_wtd       fa_elec     pro_close   hbond_sr_bb   hbond_lr_bb   hbond_bb_sc      hbond_sc     dslf_fa13         omega        fa_dun       p_aa_pp yhh_planarity           ref   rama_prepro
core.scoring.Energies: E(i)   6         -6.97          1.36          1.16          3.17          0.05         -0.12         -1.63          0.00          0.00          0.00          0.00          0.00          0.00          0.29          0.01         -1.13          0.00          2.64         -0.12

Thr8 terms, mutated structure:
------------
core.scoring.Energies: E               fa_atr        fa_rep        fa_sol  fa_intra_repfa_intra_sol_x   lk_ball_wtd       fa_elec     pro_close   hbond_sr_bb   hbond_lr_bb   hbond_bb_sc      hbond_sc     dslf_fa13         omega        fa_dun       p_aa_pp yhh_planarity           ref   rama_prepro
core.scoring.Ener

We see that there was an increase in the `fa_sol` term, which penalizes polar residues like Threonine being buried in the core:

In [210]:
fa_sol = pyrosetta.rosetta.core.scoring.ScoreType.fa_sol

V8_fa_sol = pose_relax.energies().residue_total_energies(res8)[fa_sol]
T8_fa_sol = pose_V8T.energies().residue_total_energies(res8)[fa_sol]

print(T8_fa_sol - V8_fa_sol)

2.0033733586316864


In [ ]:
# pose_from_sequence ??